In [ ]:
import sys
sys.path.append("..") # for sibling import

import torch
import walnut
import walnut.nn as nn

In [ ]:
b, c_in, c_out, h, w = 10, 1, 4, 14, 14
kernel_size = (3, 3)

X = walnut.randn((b, c_in, h, w))
W = walnut.randn((c_out, c_in, *kernel_size))
B = walnut.randn((c_out,))

t_x = torch.from_numpy(X.data).float()
t_x.requires_grad = True
t_w = torch.nn.Parameter(torch.from_numpy(W.data).float(), requires_grad=True)
t_b = torch.nn.Parameter(torch.from_numpy(B.data).float(), requires_grad=True)

strides = 1
dilation = 1
pad = "valid"

### Forward

In [ ]:
w_conv = nn.layers.Convolution2d(c_in, c_out, kernel_size=kernel_size, stride=strides, dil=dilation, pad=pad, weights=W)
w_conv.training = True
w_conv.b = B
w_out = w_conv(X)
w_out[0, 0, 0]

In [ ]:
t_conv = torch.nn.Conv2d(c_in, c_out, kernel_size, bias=False, stride=strides, dilation=dilation, padding=pad)
t_conv.weight = t_w
t_conv.bias = t_b
t_out = t_conv(t_x)
t_out[0, 0, 0]

### Backward

In [ ]:
dy = walnut.randn(w_out.shape)
t_dy = torch.from_numpy(dy.data)
x_grad = w_conv.backward(dy.data)
t_out.backward(t_dy)

X

In [ ]:
x_grad[1, 0]

In [ ]:
t_x.grad[1, 0]

W

In [ ]:
w_conv.w.grad[0, 0]

In [ ]:
t_w.grad[0, 0]

B

In [ ]:
w_conv.b.grad

In [ ]:
t_b.grad